### Import required libraries

In [257]:
import numpy as np
import pandas as pd

In [258]:
df = pd.read_excel('data/life_tables.xlsx')
df.shape

(2274, 6)

In [259]:
df.head()

,table,gender,age,qx,lx,dx
0,AT2000 (Suavizada 10%),F,0,0.001615,1000000.000000,1615.000000
1,AT2000 (Suavizada 10%),F,1,0.000680,998385.000000,678.901800
2,AT2000 (Suavizada 10%),F,2,0.000353,997706.098200,352.190253
3,AT2000 (Suavizada 10%),F,3,0.000261,997353.907947,260.309370
4,AT2000 (Suavizada 10%),F,4,0.000209,997093.598577,208.392562


In [260]:
df.columns

Index(['table', 'gender', 'age', 'qx', 'lx', 'dx'], dtype='object')

In [261]:
tables = list(df['table'].unique())

[{'value':i+1, 'label':tb} for i, tb in enumerate(tables)]

[{'label': ' AT2000 (Suavizada 10%)', 'value': 1},
 {'label': ' AT2000', 'value': 2},
 {'label': 'AT-49', 'value': 3},
 {'label': 'BR-EMSmt-v.2010', 'value': 4},
 {'label': 'BR-EMSsb-v.2010', 'value': 5},
 {'label': 'BR-EMSmt-v.2015', 'value': 6},
 {'label': 'BR-EMSsb-v.2015', 'value': 7},
 {'label': 'IBGE 2006', 'value': 8},
 {'label': 'IBGE 2008', 'value': 9},
 {'label': 'IBGE 2007', 'value': 10},
 {'label': 'IBGE 2009', 'value': 11}]

In [262]:
df_interest = pd.read_excel("data/risk_free.xlsx")

In [263]:
df_interest[df_interest['month'] == df_interest['month'].max()]['selic_year'].values[0]/100

0.0416

In [264]:
df['lx'].values.min()

0.0

In [265]:
df_ = df.query('table == " AT2000" and gender == "M"').reset_index(drop=True).copy()

In [266]:
df_['lx'].values.shape

(116,)

In [885]:
class InsuranceHandler():
    def __init__(self, df):
        self.df = df
        self.df_ = None
        self.last_i_rate_used = None
        self.max_age = None
        
        self.Dx = None
        self.Cx = None
        self.Nx = None
        self.Mx = None
        
        self.pup = None
        self.anui = None
        self.pna = None
        
        self.dif_benef = None
        self.term_benef = None 
        self.antecip_benef = None
        self.prod = None
        
        self.dif_pay = None
        self.term_pay = None
        self.antecip_pay = None
        
    def __pv_calc__(self, n):        
        '''
        This function calculates the present value factor v
        Input:
            n: period --> int or np.array
        Output:
            A float or a vector of floats with the factor (1+rate)**(-n)
        '''
        return 1/(1 + self.last_i_rate_used)**n

    def __calc_Dx__(self):
        '''
        This function calculated the Dx commutation
        Output:
              An np.array with Dx commutation
        '''
        lx = self.df_['lx'].values
        age = self.df_['age'].values
        
        return lx*self.__pv_calc__(age)
        
    def __calc_Nx__(self):
        '''
        Thist function calculated the Nx commutation
        Output:
              An np.array with Nx commutation
        '''
        return self.Dx[::-1].cumsum()[::-1]
    
    def __calc_Cx__(self):
        '''
        This function calculated the Cx commutation
        Output:
              An np.array with Cx commutation
        '''
        dx = self.df_['dx'].values
        age = self.df_['age'].values
        age_ = age + 1
        
        return dx*self.__pv_calc__(age_)
    
    def __calc_Mx__(self):
        '''
        This function calculated the Nx commutation
        Output:
              An np.array with Mx commutation
        '''
        return self.Cx[::-1].cumsum()[::-1]
    
    def __verify_prod__(self, x, n, m, antecip, prod):
        max_age = self.max_age
        
        add_one = 1 if antecip and prod == 'a' else 0
        
        m_ = 0 if m == np.inf else m
        
        criteria = x + n + m_ - add_one
    
        if prod == 'd' and n > 0:
            raise Exception('Não existe dotal puro diferido')
            
        if (prod == 'd' or prod == 'D') and m == np.inf:
            raise Exception('Não existe dotal vitalício')

        if criteria > max_age:
            raise Exception('Idade + diferimento + parazo = {} > {} idade máxima'.format(criteria, max_age))
        
    def __calc_pup__(self, dif, age, term=np.inf, antecip=True, prod='a'):

        max_age = self.max_age
        
        x = age
        n = dif
        m = term
        
        add_one = 0 if antecip and prod == 'a' else 1
        
        remove_term = 0 if term == np.inf else 1
        m_ = max_age - x - n - add_one if m == np.inf else m
        
        self.__verify_prod__(x, n, m, antecip, prod)
        
        if prod == "D":
            pup = (self.Mx[x + n] - self.Mx[x + n + m_] + self.Dx[x + m_]) / \
                    self.Dx[x]
        elif prod == "d":
            pup = self.Dx[x + m_] / self.Dx[x]
        elif prod == "A":
            pup = (self.Mx[x + n] - remove_term*self.Mx[x + n + m_]) / \
                    self.Dx[x]
        elif prod == "a":
            pup = (self.Nx[x + n + add_one] - remove_term*self.Nx[x + n + m_ + add_one]) / \
                     self.Dx[x]     
        return pup
    
    def __calc_prov_retro__(self, t, x):
        max_x = self.max_age
        n = self.dif_benef
        m = self.term_benef
        i = self.dif_pay
        k = self.term_pay
        prod = self.prod
        pay_antecip = self.antecip_pay
        benef_antecip = self.antecip_benef
        
        adjust_pay = 1 if pay_antecip else 0
        adjust_benef = 1 if benef_antecip and prod == 'a' else 0

        P = self.pna
        E = 1/self.__calc_pup__(0, x, t, pay_antecip, 'd')
        print(E)
        #payment
        if  0 < t <= i - adjust_pay:
            a = 0
        elif i - adjust_pay < t: 
            a = self.__calc_pup__(0, x, min(t, k), pay_antecip, 'a')
        elif t == 0:
            a = 0
            
        #benefit     
        if 0 < t <= n - adjust_benef:
            A = 0
        elif n - adjust_benef < t:
            print(x+n, x+t)
            A = self.__calc_pup__(n, x, min(t-n,m), benef_antecip, prod)
        elif t == 0:
            A = 0
        
        print(A)
        print(a)
        V = (P*a - A)*E
        return V

    def __calc_prov_prosp__(self, t, x):
        
        max_x = self.max_age
        n = self.dif_benef
        m = self.term_benef
        i = self.dif_pay
        k = self.term_pay
        prod = self.prod
        pay_antecip = self.antecip_pay
        benef_antecip = self.antecip_benef
        
        adjust_pay = 1 if pay_antecip else 0
        adjust_benef = 1 if benef_antecip and prod == 'a' else 0

        P = self.pna

        #payment
        if 0 < t <= i - adjust_pay:
            a = self.__calc_pup__(max(i-t, 0), x+t, k, pay_antecip, 'a')
        elif i - adjust_pay < t <= k - adjust_pay:
            a = self.__calc_pup__(0, x+t, max(i+k-t,0), pay_antecip, 'a')
        elif t > k - adjust_pay or t == 0:
            a = 0
         
        #benefit     
        if 0 < t <= n - adjust_benef:
            A = self.__calc_pup__(max(n-t,0), x+t, m, benef_antecip, prod)
        elif n - adjust_benef < t <= n + m - adjust_benef:
            print(x+t + n + m-t)
            A = self.__calc_pup__(0, x+t, max(n + m - t,0), benef_antecip, prod)
        elif t > n + m - adjust_benef or t == 0:
            A = 0
        print(A)
        print(a)
        V = A - P*a
        return V
     
    def select_table(self, table, gender):
        query_string = 'table == "{}" and gender == "{}"'.\
                                          format(table, gender)
        self.df_ = self.df.query(query_string).\
                                          reset_index(drop=True).copy()
        self.max_age = self.df_['age'].max()
        
    def gen_commutations(self, i_rate):
        
        if self.df_ is None:
            raise Exception('Life table must be filtered')
            
        self.last_i_rate_used = i_rate
        
        self.Dx = self.__calc_Dx__()
        self.Nx = self.__calc_Nx__()
        self.Cx = self.__calc_Cx__()
        self.Mx = self.__calc_Mx__()
              
    def calc_premium(self, age, dif_benef=0, term_benef=np.inf, 
                     antecip_benef=True, prod='a', 
                     dif_pay=0, term_pay=np.inf, antecip_pay=True):
        
        self.pup = self.__calc_pup__(dif_benef, age, term_benef, 
                                     antecip_benef, prod)
        self.dif_benef = dif_benef
        self.term_benef = term_benef 
        self.antecip_benef = antecip_benef
        self.prod = prod
        
        self.anui = self.__calc_pup__(dif_pay, age, term_pay, antecip_pay, "a")
        self.dif_pay = dif_pay
        self.term_pay = term_pay
        self.antecip_pay = antecip_pay
        
        self.pna = self.pup / self.anui

In [886]:
a = InsuranceHandler(df)
a.select_table(" AT2000", "M")
a.gen_commutations(0.1)

In [894]:
a.calc_premium(age=40, dif_benef=10, term_benef=20, dif_pay=5, term_pay=3, prod='A')

In [895]:
a.anui

1.6839105090758815

In [896]:
a.pup

0.019035692751248445

In [897]:
a.pna

0.011304456293045587

In [898]:
a.__calc_prov_prosp__(4,40)

0.028011012520373427
2.477873485839879


0.0

In [899]:
a.__calc_prov_retro__(4,40)

1.4714995081298705
0
0


0.0

In [893]:
'D'== 'd'

False

In [662]:
(a.Nx[40] - a.Nx[53])/a.Dx[40]

7.753883059311794

In [ ]:
    def __calc_pup__(self, dif, term=np.inf, antecip=True, prod='a'):
        '''
        This function calcultes the net sigle premium of an Annuity, Insurance or Endowment

        Input: 
               dif: deferment period --> int
               term: product term --> int
               antecip: True if the annuity is anticipated --> Boolean
               prod: A, a, D for Insurance, annuity, Endowment
               
        Output: 
              A vector np.array of net sigle premiums --> np.array
        '''
        
        if prod == 'A' or prod == 'D' or prod == 'd':
            NMx = self.Mx.copy()
        else:
            NMx = self.Nx.copy()
        
        Dx = self.Dx.copy()
        Cx = self.Cx.copy()
        
        #Antecip or postecip, for insurance it is always post
        add_one = 0 if antecip == True or prod == 'A' else 1
        
        #Copes with the differences between whole life and term products
        if term == np.inf:
            ini_ = dif + add_one if dif + add_one > 0 else None
            ini_sign = ini_ if ini_ is None else - ini_ 
            #Insurance and annuities
            #(Mx+n) / Dx or (Nx+n) / Dx, n = 0,1,2,3,...
            result = NMx[ini_:] / Dx[:ini_sign] 
        else:
            #Copes with deferment period
            if prod == 'd':
                dif = 0
                add_one = 0
            elif prod == 'D':
                #Deferred Endowment
                add_one = 0

            #Avoid a dif + term greater than life table's size
            term_ = min(term, NMx.shape[0] - 1 - dif - add_one)
            ini_dif = dif + term_ + add_one
            ini_term = dif + add_one
            len_ = NMx[ini_dif:].shape[0]
            fin_term = ini_term + len_

            if prod == 'D':
                #Endowment
                #(Mx+m - Mx+m + Dx+m) / Dx
                result = (NMx[ini_term:fin_term] - NMx[ini_dif:] + Dx[ini_dif:]) / Dx[:len_]
            elif prod == 'd':
                #Endowment
                #(Dx+m) / Dx
                result = Dx[ini_dif:] / Dx[:len_]
            else:
                #Insurance and annuities
                #(Mx+n - Mx+n+m) / Dx or (Nx+n - Nx+n+m) / Dx, n = 0,1,2,3,...
                result = (NMx[ini_term:fin_term] - NMx[ini_dif:]) / Dx[:len_]

        return result
